In [82]:
# Created by Medad Rufus Newman on 19/03/2020
# This program localises the position of a High altitude balloon without the use of the GPS by finding the 
# center of the minimum bounding circle of gateways that pick up the packets. The program then estimates altitude assuming the line of sight
# horizon is the limit of the radio's range
#
# Occasionally, the GPS on the balloons fail, as it has done on ICSPACE19 but the radio is still transmitting.
# We try to locate it based on where the radio transmissions are picked up. Note that ICSPACE is a floater balloon
# designed to travel around the world.
#
# This program takes the csv data of a balloon flight, specifically ICSPACE19, from habitat by exporting as a CSV. 
# Then it matches the receive gateways with their location in a lookup table provided by The Things Network.

import pandas as pd
import requests
from pandas.io.json import json_normalize

In [83]:
# Download latest balloon data from habhub
balloon_ept_url = "http://habitat.habhub.org/habitat/_design/ept/_list/csv/payload_telemetry/flight_payload_time?include_docs=true&startkey=[%2299a403a8b8a8fbaa9de2332bcbf9605e%22,%221260d7d8adf6d27c5380a641ca7d786a%22]&endkey=[%2299a403a8b8a8fbaa9de2332bcbf9605e%22,%221260d7d8adf6d27c5380a641ca7d786a%22,[]]&fields=_sentence,_receivers,sentence_id,time,pressure_mbar,temperature_external,latitude,longitude,altitude,solar,satellites"
r = requests.get(balloon_ept_url, allow_redirects=True)
open('flight_payload_time.csv', 'wb').write(r.content)

62115

In [84]:
# Read cached balloon data from habhub
habub_df = pd.read_csv('flight_payload_time.csv')
habub_df['time'] = pd.to_datetime(habub_df['time'], format="%H:%M:%S")

habub_df.head()

,_sentence,_receivers,sentence_id,time,pressure_mbar,temperature_external,latitude,longitude,altitude,solar,satellites
0,"$$icspace19,3,09:12:08,545.9,-263.7,51.5098,-0...","eui-58a0cbfffe800c8a,eui-fcc23dfffe0a7ef5,eui-...",3,1900-01-01 09:12:08,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
1,"$$icspace19,3,09:12:09,545.9,-263.7,51.5098,-0...","eui-7276ff000b03239e,eui-7276ff000b03224d,eui-...",3,1900-01-01 09:12:09,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
2,"$$icspace19,3,09:12:10,545.9,-263.7,51.5098,-0...","eui-7276ff000b032627,eui-7276ff000b032390,eui-...",3,1900-01-01 09:12:10,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
3,"$$icspace19,4,09:58:53,545.9,-263.7,51.5098,-0...","eui-fcc23dfffe0b89c8,eui-b827ebfffe0e1b64,eui-...",4,1900-01-01 09:58:53,545.9,-263.7,51.5098,-0.118,0.0,2.44,0
4,"$$icspace19,4,09:58:54,545.9,-263.7,51.5098,-0...","eui-7276ff000b03241e,eui-7276ff000b0325cd,eui-...",4,1900-01-01 09:58:54,545.9,-263.7,51.5098,-0.118,0.0,2.44,0


In [85]:
# Process data from Habhub, making a dict of all the transmissions and the gateways that picked up the telemetry
# But we don't know the locations of the gateways yet
import datetime

all_transmissions  = {}

done_first_row = False
sentence_counter = 1
current_transmission = None
working_time  = None

# WARNING: there appears to be one extra tx being counted. Need to test further

for index, row in habub_df.iterrows():
    
    if not done_first_row:
        working_time = row["time"]
        done_first_row = True
        
        all_transmissions["transmission_"+str(sentence_counter)] = {"receivers":row["_receivers"].split(',')}
        current_transmission = "transmission_"+str(sentence_counter)
        sentence_counter +=1
        continue
    else:
        current_time = row["time"]
        delta_time = current_time - working_time
        working_time = current_time
        
        #print(delta_time)
        if delta_time< datetime.timedelta(seconds=5):
            # append to last tx object
            all_transmissions[current_transmission]["receivers"] += row["_receivers"].split(',')
        else:
            # make a new tx object
            current_tx  = all_transmissions["transmission_"+str(sentence_counter)] = {"receivers":row["_receivers"].split(',')}
            current_transmission = "transmission_"+str(sentence_counter)
            sentence_counter +=1

In [86]:
# Here we get all the gateways around the world and extract their ID and longitude and latitude

# read all gateway data
df_gateway_data  = pd.read_json("gateway-data.json")


df_gateway_data = df_gateway_data.T
#df_gateway_data.head()



# And in another column the df_gateway_data containing long and lat for each gateway
import numpy as np
latitudes = []
longitudes = []



for index, row in df_gateway_data.iterrows():
    if pd.notna(row["location"]):
        latitudes.append(row["location"]["latitude"])
        longitudes.append(row["location"]["longitude"])

    else:
        latitudes.append(np.nan)
        longitudes.append(np.nan)
        

df_gateway_data["latitude"] = latitudes
df_gateway_data["longitude"] = longitudes

In [87]:
# Now create the lookup table from our flight data. We match the ids of the gateways that received packets from our balloon and forwarded
# to Habhub. We lookup the locations of the gateways using data from The Things Network

txs = {}

for k in all_transmissions:
    print()
    # Iterate through all the receivers and find their coordinates
    print(k,"was received by",len(all_transmissions[k]["receivers"]), "gateways")
    
    longs = []
    lats =[]
    receivers = []
    for receiver in all_transmissions[k]["receivers"]:
        lookup_row  = df_gateway_data.loc[df_gateway_data['id'] == receiver]
        if lookup_row.empty:
            continue
        #print(receiver,lookup_row["id"].values,lookup_row["longitude"].values,lookup_row["latitude"].values)
        #print(lookup_row["latitude"].values[0])
        
        if not np.isnan(lookup_row["latitude"].values[0]):
            receivers.append(receiver)
            lats.append(lookup_row["latitude"].values[0])
            longs.append(lookup_row["longitude"].values[0])
    
    txs[k] = {"receivers":receivers,"longitudes":longs,"latitudes":lats}





transmission_1 was received by 62 gateways

transmission_2 was received by 67 gateways

transmission_3 was received by 14 gateways

transmission_4 was received by 74 gateways

transmission_5 was received by 1 gateways

transmission_6 was received by 85 gateways

transmission_7 was received by 9 gateways

transmission_8 was received by 119 gateways

transmission_9 was received by 113 gateways

transmission_10 was received by 1 gateways

transmission_11 was received by 87 gateways

transmission_12 was received by 18 gateways

transmission_13 was received by 38 gateways

transmission_14 was received by 1 gateways

transmission_15 was received by 1 gateways

transmission_16 was received by 28 gateways

transmission_17 was received by 48 gateways

transmission_18 was received by 50 gateways

transmission_19 was received by 53 gateways

transmission_20 was received by 1 gateways

transmission_21 was received by 38 gateways

transmission_22 was received by 1 gateways

transmission_23 was rec

In [88]:
# Now time to find the bounding circle around the coordinates for all transission.
# 
# Smallest enclosing circle - Library (Python)
# 
# Copyright (c) 2018 Project Nayuki
# https://www.nayuki.io/page/smallest-enclosing-circle
# 
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
# 
# You should have received a copy of the GNU Lesser General Public License
# along with this program (see COPYING.txt and COPYING.LESSER.txt).
# If not, see <http://www.gnu.org/licenses/>.
# 

import math, random


# Data conventions: A point is a pair of floats (x, y). A circle is a triple of floats (center x, center y, radius).

# Returns the smallest circle that encloses all the given points. Runs in expected O(n) time, randomized.
# Input: A sequence of pairs of floats or ints, e.g. [(0,5), (3.1,-2.7)].
# Output: A triple of floats representing a circle.
# Note: If 0 points are given, None is returned. If 1 point is given, a circle of radius 0 is returned.
# 
# Initially: No boundary points known
def make_circle(points):
    # Convert to float and randomize order
    shuffled = [(float(x), float(y)) for (x, y) in points]
    random.shuffle(shuffled)
    
    # Progressively add points to circle or recompute circle
    c = None
    for (i, p) in enumerate(shuffled):
        if c is None or not is_in_circle(c, p):
            c = _make_circle_one_point(shuffled[ : i + 1], p)
    return c


# One boundary point known
def _make_circle_one_point(points, p):
    c = (p[0], p[1], 0.0)
    for (i, q) in enumerate(points):
        if not is_in_circle(c, q):
            if c[2] == 0.0:
                c = make_diameter(p, q)
            else:
                c = _make_circle_two_points(points[ : i + 1], p, q)
    return c


# Two boundary points known
def _make_circle_two_points(points, p, q):
    circ = make_diameter(p, q)
    left  = None
    right = None
    px, py = p
    qx, qy = q
    
    # For each point not in the two-point circle
    for r in points:
        if is_in_circle(circ, r):
            continue
        
        # Form a circumcircle and classify it on left or right side
        cross = _cross_product(px, py, qx, qy, r[0], r[1])
        c = make_circumcircle(p, q, r)
        if c is None:
            continue
        elif cross > 0.0 and (left is None or _cross_product(px, py, qx, qy, c[0], c[1]) > _cross_product(px, py, qx, qy, left[0], left[1])):
            left = c
        elif cross < 0.0 and (right is None or _cross_product(px, py, qx, qy, c[0], c[1]) < _cross_product(px, py, qx, qy, right[0], right[1])):
            right = c
    
    # Select which circle to return
    if left is None and right is None:
        return circ
    elif left is None:
        return right
    elif right is None:
        return left
    else:
        return left if (left[2] <= right[2]) else right


def make_diameter(a, b):
    cx = (a[0] + b[0]) / 2.0
    cy = (a[1] + b[1]) / 2.0
    r0 = math.hypot(cx - a[0], cy - a[1])
    r1 = math.hypot(cx - b[0], cy - b[1])
    return (cx, cy, max(r0, r1))


def make_circumcircle(a, b, c):
    # Mathematical algorithm from Wikipedia: Circumscribed circle
    ox = (min(a[0], b[0], c[0]) + max(a[0], b[0], c[0])) / 2.0
    oy = (min(a[1], b[1], c[1]) + max(a[1], b[1], c[1])) / 2.0
    ax = a[0] - ox;  ay = a[1] - oy
    bx = b[0] - ox;  by = b[1] - oy
    cx = c[0] - ox;  cy = c[1] - oy
    d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
    if d == 0.0:
        return None
    x = ox + ((ax*ax + ay*ay) * (by - cy) + (bx*bx + by*by) * (cy - ay) + (cx*cx + cy*cy) * (ay - by)) / d
    y = oy + ((ax*ax + ay*ay) * (cx - bx) + (bx*bx + by*by) * (ax - cx) + (cx*cx + cy*cy) * (bx - ax)) / d
    ra = math.hypot(x - a[0], y - a[1])
    rb = math.hypot(x - b[0], y - b[1])
    rc = math.hypot(x - c[0], y - c[1])
    return (x, y, max(ra, rb, rc))


_MULTIPLICATIVE_EPSILON = 1 + 1e-14

def is_in_circle(c, p):
    return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) <= c[2] * _MULTIPLICATIVE_EPSILON


# Returns twice the signed area of the triangle defined by (x0, y0), (x1, y1), (x2, y2).
def _cross_product(x0, y0, x1, y1, x2, y2):
    return (x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0)


In [89]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [90]:
# Now calculate and record down the centers of the minimum bounding circle of all gateways that pick up the
# transmissions

import csv

with open('coords.csv', mode='w') as coords_file:
    writer = csv.writer(coords_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(["lat","long","alt"])

    for i in all_transmissions:
    
        lat,long,radius_degrees = make_circle(zip(txs[i]["latitudes"],txs[i]["longitudes"]))
        radius_meters = haversine(radius_degrees,0,0,0)
        calculated_altitude  = radius_meters**2/12.756 # Taken from https://github.com/rossengeorgiev/habitat-mobile-tracker/blob/f4e4bfbd225962f9e875e5c45d350ac840762108/js/tracker.js#L895
        print("calculated coordinates of balloon: lat:{0:.5f} long:{1:.5f} alt: {2:.0f}".format(lat,long,calculated_altitude))
        writer.writerow([lat,long,calculated_altitude])




calculated coordinates of balloon: lat:49.86532 long:9.09510 alt: 6414
calculated coordinates of balloon: lat:49.23669 long:9.14828 alt: 8720
calculated coordinates of balloon: lat:48.46155 long:9.38233 alt: 358
calculated coordinates of balloon: lat:49.04214 long:8.95128 alt: 7203
calculated coordinates of balloon: lat:47.27012 long:9.60597 alt: 0
calculated coordinates of balloon: lat:49.11190 long:9.25982 alt: 8556
calculated coordinates of balloon: lat:48.57799 long:9.42236 alt: 343
calculated coordinates of balloon: lat:49.97707 long:9.77413 alt: 8408
calculated coordinates of balloon: lat:49.98863 long:9.74012 alt: 8552
calculated coordinates of balloon: lat:48.61354 long:8.84790 alt: 0
calculated coordinates of balloon: lat:49.08464 long:9.82257 alt: 10826
calculated coordinates of balloon: lat:49.37281 long:11.74854 alt: 7775
calculated coordinates of balloon: lat:49.12168 long:11.83659 alt: 7949
calculated coordinates of balloon: lat:48.68800 long:9.03374 alt: 0
calculated coo

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# KML generator to visualise the path on Google maps

header = """
<?xml version="1.0" encoding="UTF-8"?><kml xmlns="http://www.opengis.net/kml/2.2">         <Document>
<Placemark><name>test</name>
<description>test Desc</description>
<LineString>
<coordinates>
"""

"""
-80.54400115,43.4250264
-80.52674314,43.43127701
-80.5274517,43.43458707
-80.53223781,43.43876923
-80.54385782,43.44993036
-80.53949137,43.45723788
-80.53950793,43.46780893
-80.53352615,43.4730443
-80.53491389,43.47816267
-80.54136061,43.48417145
-80.54163034,43.48439869
"""
end = """
</coordinates>
</LineString>
</Placemark></Document>
</kml> 
"""

with open("coords_path.kml", "w") as file:
    file.write(header)
    
    for i in txs:
        lat,long,rad = make_circle(zip(txs[i]["latitudes"],txs[i]["longitudes"]))
        file.write("{0:+.5f},{1:+.5f}\n".format(long,lat))
    
    file.write(end)


